In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

driver = webdriver.Chrome()

In [2]:
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import numpy as np
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
print(selenium.__version__)

4.15.2


In [7]:
chrome_options = Options()

chrome_options.add_experimental_option("detach", True)
chrome_options.add_experimental_option("excludeSwitches",["enable-logging"])

driver = webdriver.Chrome(options=chrome_options)

In [8]:
musinsa_url = "https://www.musinsa.com/app/codimap/lists"
driver.get(musinsa_url)
driver.maximize_window()

In [9]:
reviews_button = driver.find_element(By.XPATH, '/html/body/div[3]/div[2]/form/div[4]/div/div[1]/a[3]/span') # 일요일 웹툰 버튼 클릭
reviews_button.click()

In [23]:
driver = webdriver.Chrome(options=chrome_options)

musinsa_list = []
musinsa_url = "https://www.musinsa.com/app/codimap/lists"
driver.get(musinsa_url)
driver.implicitly_wait(2)

try:
    driver.maximize_window()
except:
    pass

for web_page in range(2):

    interval = 1 

    prev_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")

        time.sleep(interval)

        curr_height = driver.execute_script("return document.body.scrollHeight")
        if curr_height == prev_height:
            break

        prev_height = curr_height

    columns = ['codimap_category','codimap_title','codimap_date','views','comment_count']   
    values = []                                           
    soup = BeautifulSoup(driver.page_source, 'html.parser')    
    data_rows = soup.find_all('li', attrs={'class':'style-list-item'}) 

    for i, row in enumerate(data_rows):                     

        blank = []

        codimap_category = row.find('span', attrs={'class':'style-list-information__text'})     
        if codimap_category:                                                                        
            codimap_cat_text = codimap_category.get_text().strip()                                    
            blank.append(codimap_cat_text)
        else:                                                              
            blank.append('Something went wrong with processing the category')
            continue


        codimap_title = row.find('strong', attrs={'class':'style-list-information__title'})     
        if codimap_title:
            codimap_title = codimap_title.get_text().strip()
            blank.append(codimap_title)
        else:
            blank.append('Something went wrong with processing the title')
            continue

        # Find all span elements within the list
        span_elements = row.find_all('span', class_='post-information__text')
        
        # skipping if 'post-information__text--new' is present in the span elements
        if 'post-information__text--new' in span_elements[0]['class']:
            continue

        codimap_date = span_elements[0].text.strip()
        if codimap_date:
            blank.append(codimap_date)
        else:
            blank.append('Something went wrong with processing the date')
            continue
        
        views = span_elements[1].text.strip()
        if views:
            blank.append(views)
        else:
            blank.append('Something went wrong with processing the views')
            continue

        comment_count = span_elements[2].text.strip()
        if comment_count:
            blank.append(comment_count)
        else:
            blank.append('Something went wrong with processing the comment_count')
            continue

        values.append(blank)

    reviews_button = driver.find_element(By.XPATH, '/html/body/div[3]/div[2]/form/div[4]/div/div[1]/a[3]/span') # highest reviews button
    reviews_button.click()
    time.sleep(1)

    df = pd.DataFrame(values, columns = columns)
    musinsa_list.append(df)


In [24]:
df_ = pd.concat(musinsa_list).reset_index(drop=True) 
df_.head(10)

,codimap_category,codimap_title,codimap_date,views,comment_count
0,캐주얼,톤온톤 코디,21.07.02,"조회 17,320",댓글 50
1,캐주얼,블랙 매니아,21.02.23,"조회 36,357",댓글 47
2,캐주얼,오늘의 코디,22.03.22,"조회 13,427",댓글 44
3,댄디,추천 댄디 룩,21.09.15,"조회 21,623",댓글 32
4,캐주얼,릴랙스 룩,21.02.23,"조회 24,053",댓글 25
5,스트릿,힙하다 힙해,21.02.03,"조회 36,109",댓글 24
6,댄디,데이트 코디,21.12.24,"조회 10,997",댓글 19
7,캐주얼,심플한 코디,21.07.30,"조회 41,376",댓글 18
8,댄디,훈훈 남친 룩,21.03.24,"조회 20,025",댓글 17
9,댄디,서머 데이트 룩,21.07.26,"조회 8,373",댓글 17


In [25]:
df_.shape

(60, 5)

In [26]:
df_.to_csv('YBIGTA_24기_Jae_Min_Hong_web_crawling.csv', encoding = 'utf-8-sig')